# Projeto Grupo BT-G3


<center><img src="https://www.inteli.edu.br/wp-content/uploads/2021/08/20172028/marca_1-2.png" width="50%" height="50%"/></center>

<h1 align='center'><b>IA para Marketing: Monitoramento de campanhas utilizando processamento de linguagem natural (PLN)<b></h1>

<center><img src="https://upload.wikimedia.org/wikipedia/commons/c/c2/Btg-logo-blue.svg" width="50%" height="50%"/></center>

<h2 align='center'>O Banco BTG Pactual enfrenta um desafio na área de Marketing em entender as necessidades e demandas dos clientes de maneira fácil e rápida nas redes sociais. A solução proposta para esse problema foi o desenvolvimento de uma Inteligência Artificial utilizando processamento de linguagem natural (PLN), capaz de monitorar as campanhas de marketing, voltadas para o Instagram. O objetivo principal dessa solução é rastrear os dados em tempo real, analisar e interpretar as mensagens e comentários enviados pelos clientes na rede social, a fim de identificar as necessidades e demandas de forma precisa e eficiente.</h2>

---

# Sobre os dados

Esse projeto está utilizando dados coletados e tratados pela equipe de Automation do BTG Pactual, o qual disponibilizou o dataset. Com base nas informações dispostas nesse dataset, realizaremos insights a cerca dos comentários feitos nos posts do Instagram do próprio banco. Vale lembrar que os dados estão anonimizados e resguardados para manter a privacidade e ética com os usuários e com o banco.

# 1. Instalação / Setup

Para o início do projeto, fizemos o desenvolvimento no Google Colab, por isso temos uma célula de conexão com o Google Drive, para poder acessar os dados. Caso seja rodado no Jupyter Notebook, precisará do dataset baixado.

In [ ]:
#Conectar com o Google Drive

from google.colab import drive
drive.mount('/content/drive')

#Conectando o ambiente ao Google Drive

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Aqui nós fazemos as importações para tratamento, pré-processamento e modelamento do Bag of Words. 

In [ ]:
!pip install wordcloud

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from wordcloud import WordCloud

import re

from sklearn.feature_extraction.text import CountVectorizer

import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('rslp')
nltk.download('wordnet')
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import PlaintextCorpusReader
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk import tokenize
from nltk import word_tokenize, pos_tag

!pip install emoji
import emoji

import spacy
!pip install -U spacy
!python -m spacy download pt_core_news_sm

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Package rslp is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
2023-05-23 20:47:09.895980: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 67.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')


Rodando o dataset, para analisar seu conteúdo:

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Módulo 6/Base 2.csv')
data.head(5)

,id,"""dataPublicada""","""autor""","""texto""","""sentimento""","""tipoInteracao""","""anomalia""","""probabilidadeAnomalia""","""linkPost""","""processado""","""contemHyperlink"""
0,1,"""2022-03-04 09:38:00""",v8_capital,Confira os resultados dos nossos fundos no mês...,NEUTRAL,marcação,1,52,https://www.instagram.com/p/CarqO6drRux/,0,0
1,2,"""2022-03-04 09:29:00""",winthegame_of,A Alvarez & Marsal estará conosco no Sportainm...,NEUTRAL,marcação,0,43,https://www.instagram.com/p/CarpQUluh7z/,0,0
2,3,"""2022-03-04 09:20:00""",marta_bego,#Repost btgpactual with make_repost ・・・ Entend...,NEUTRAL,marcação,0,45,https://www.instagram.com/p/CaroK9Nu8tC/,0,0
3,4,"""2022-03-04 08:36:00""",lmviapiana,Minuto touro de ouro,POSITIVE,comentário,0,38,https://www.instagram.com/p/CapXhB5Lvas/#17977...,0,0
4,5,"""2022-03-03 21:10:00""",vanilson_dos,@ricktolledo Sim,NEUTRAL,resposta,0,17,https://www.instagram.com/p/CapXhB5Lvas/#17842...,0,0


# 2. Tratamento dos Dados

No tratamento de dados, nós analisamos o que podemos mudar no dataframe para fazer uma análise melhor de seus dados e alcançar o objetivo traçado. Dessa forma, é possível visualizar algumas etapas do tratamento abaixo, por exemplo:
- Renomear as colunas para retirar aspas duplos;
- Analisar as colunas com objetos que possam ser quantitativos;
- Remover algumas colunas que não servirão para o desenvolvimento do projeto;
- Filtro para analisar comentários que não tenham o 'btgpactual' como autor, para ter uma análise somente dos usuários.

In [ ]:
data = data.rename(columns={'"anomalia"' : 'anomalia', '"dataPublicada"' : 'dataPublicada', '"autor"' : 'autor', '"texto"' : 'texto', '"sentimento"' : 'sentimento', '"tipoInteracao"' : 'tipoInteracao', '"probabilidadeAnomalia"' : 'probabilidadeAnomalia', '"linkPost"' : 'linkPost', '"processado"' : 'processado',  '"contemHyperlink"' : 'contemHyperlink' })

In [ ]:
data.head(3)

,id,dataPublicada,autor,texto,sentimento,tipoInteracao,anomalia,probabilidadeAnomalia,linkPost,processado,contemHyperlink
0,1,"""2022-03-04 09:38:00""",v8_capital,Confira os resultados dos nossos fundos no mês...,NEUTRAL,marcação,1,52,https://www.instagram.com/p/CarqO6drRux/,0,0
1,2,"""2022-03-04 09:29:00""",winthegame_of,A Alvarez & Marsal estará conosco no Sportainm...,NEUTRAL,marcação,0,43,https://www.instagram.com/p/CarpQUluh7z/,0,0
2,3,"""2022-03-04 09:20:00""",marta_bego,#Repost btgpactual with make_repost ・・・ Entend...,NEUTRAL,marcação,0,45,https://www.instagram.com/p/CaroK9Nu8tC/,0,0


In [ ]:
data.describe()

,id,anomalia,probabilidadeAnomalia,processado,contemHyperlink
count,1.235500e+04,12355.000000,12355.000000,12355.0,12355.000000
mean,1.757789e+06,0.222339,37.325698,0.0,0.005423
std,7.967971e+05,0.415835,16.374936,0.0,0.073443
min,1.000000e+00,0.000000,7.000000,0.0,0.000000
25%,1.314377e+06,0.000000,24.000000,0.0,0.000000
50%,1.783879e+06,0.000000,37.000000,0.0,0.000000
75%,2.388714e+06,0.000000,48.000000,0.0,0.000000
max,3.045012e+06,1.000000,100.000000,0.0,1.000000


In [ ]:
data_dropado = data.drop(['processado', 'id', "anomalia", "probabilidadeAnomalia", "contemHyperlink", "linkPost"], axis=1)
data_dropado.head(3)

,dataPublicada,autor,texto,sentimento,tipoInteracao
0,"""2022-03-04 09:38:00""",v8_capital,Confira os resultados dos nossos fundos no mês...,NEUTRAL,marcação
1,"""2022-03-04 09:29:00""",winthegame_of,A Alvarez & Marsal estará conosco no Sportainm...,NEUTRAL,marcação
2,"""2022-03-04 09:20:00""",marta_bego,#Repost btgpactual with make_repost ・・・ Entend...,NEUTRAL,marcação


In [ ]:
data_limpo = data_dropado.loc[data_dropado['autor'] != 'btgpactual']
data_limpo

,dataPublicada,autor,texto,sentimento,tipoInteracao
0,"""2022-03-04 09:38:00""",v8_capital,Confira os resultados dos nossos fundos no mês...,NEUTRAL,marcação
1,"""2022-03-04 09:29:00""",winthegame_of,A Alvarez & Marsal estará conosco no Sportainm...,NEUTRAL,marcação
2,"""2022-03-04 09:20:00""",marta_bego,#Repost btgpactual with make_repost ・・・ Entend...,NEUTRAL,marcação
3,"""2022-03-04 08:36:00""",lmviapiana,Minuto touro de ouro,POSITIVE,comentário
4,"""2022-03-03 21:10:00""",vanilson_dos,@ricktolledo Sim,NEUTRAL,resposta
...,...,...,...,...,...
12350,"""2022-11-30 11:51:00""",onixcapital,Uma noite de encontros e muito conhecimento on...,NEUTRAL,marcação
12351,"""2022-11-30 12:25:00""",amgcapitalinvest,Erros financeiros para eliminar antes dos 30 a...,NEUTRAL,marcação
12352,"""2022-11-30 12:18:00""",jayipacderota,Estou muito grato por todo o esforço que você ...,POSITIVE,comentário
12353,"""2022-11-30 12:50:00""",moinho_cultural,"""👉GUADAKAN está chegando!!!! A aula pública e...",NEUTRAL,marcação


# 3. Testes isolados



Nesta seção, começamos a fazer testes isolados para demonstrar como é o funcionamento de alguns dos pré-processos que iremos utilizar no dataframe principal. O objetivo é dar um exemplo para visualizar como iremos utilizar antes de ser aplicado em outras funções. Os pré-processos são:
- Tokenização
- Remoção de Stopwords
- Remoção de Alfanuméricos
- Stemming

#### 3.1 Tokenização

In [ ]:
# Comentar...

text = "Minuto touro de ouro! 👍"

words = tokenize.word_tokenize(text, language='portuguese')

words

['Minuto', 'touro', 'de', 'ouro', '!', '👍']

#### 3.2 Tratamento de emoji

In [ ]:
demojized = [emoji.demojize(word) if emoji.emoji_count(word) > 0 else word for word in words]
demojized = [word.replace(":", "").replace("_", "") if any(c in word for c in [":", "_"]) else word for word in demojized]

demojized

['Minuto', 'touro', 'de', 'ouro', '!', 'thumbsup']

#### 3.3 Remoção de stopwords

In [ ]:
stopwords = nltk.corpus.stopwords.words('portuguese')

stopwords[:10]

['a',
 'à',
 'ao',
 'aos',
 'aquela',
 'aquelas',
 'aquele',
 'aqueles',
 'aquilo',
 'as']

In [ ]:
filtered_demojized = []

for word in demojized:
    if word not in stopwords:
        filtered_demojized.append(word)

filtered_demojized

['Minuto', 'touro', 'ouro', '!', 'thumbsup']

#### 3.4 Remoção de Alfanuméricos

In [ ]:
frasezinha = ' '.join(filtered_demojized)
nova_frase = re.sub(r'[^\w\s]', '', frasezinha)
frasezinha = nova_frase.strip()
print(frasezinha)

Minuto touro ouro  thumbsup


#### 3.5 Lematização

# 4. Definição de funções


Aqui em definição de funções é onde iremos criar as funções referentes aos tratamentos que mostramos acima! Já que, dessa forma, poderemos chamar essas mesmas funções em qualquer momento para fazer o tratamento do conteúdo textual.

#### 4.1 Tokenização

In [ ]:
def tokenizer(comment):
    if isinstance(comment, str):
        tokens = nltk.word_tokenize(comment)
        return tokens
    else:
        return []

#### 4.2 Tratamento de emoji

#### 4.3 Remoção de stopwords

In [ ]:
def remove_stopwords(tokens):
    if isinstance(tokens, list): 
        comments_filtered = []
        for token in tokens:
          tk = token.lower()
          if tk not in stopwords:
              comments_filtered.append(tk)
        return comments_filtered
    else:
        return []

#### 4.4 Remoção de acentos


In [ ]:
def remover_pontuacao(tokens):
    tokens_sem_pontuacao = []
    for token in tokens:
        token_sem_pontuacao = re.sub(r'[^\w\s]', '', token)
        if token_sem_pontuacao != '':
            tokens_sem_pontuacao.append(token_sem_pontuacao)
    return tokens_sem_pontuacao

#### 4.5 Lematização


Quando a biblioteca é utilizada sem especificar

In [ ]:
def lemmatize_text(text_list):
    # Carregar o modelo pré-treinado do SpaCy para o idioma português
    nlp = spacy.load("pt_core_news_sm")
    
    lemmatized_texts = []
    
    for text in text_list:
        # Processar o texto com o modelo do SpaCy
        doc = nlp(text)
        
        # Lemmatizar cada token no texto
        lemmas = [token.lemma_ for token in doc]
        
        # Adicionar os lemas a uma lista de textos lematizados
        lemmatized_texts.append(" ".join(lemmas))
    
    return lemmatized_texts


# 5. Testes das funções

Aqui nos testes das funções, iremos aplicar casos específicos para visualizar como nossa função se comporta, para, logo depois, aplica-la no dataframe principal

In [ ]:
#nos testes das funções devemos colocar oq deu certo e oq deu errado (ex: tirar os emotes, não tirar palavras como 'pra', etc)

#### 5.1 Tokenização
- Para o processo de tokenização, é possível visualizar que em qualquer caso, a função consegue tokenizar todo e qualquer termo existente da frase.

In [ ]:
teste_tokenizacao = "Estamos fazendo um projeto pro BTG! Ele é muito interessante."
teste_ja_tokenizado = tokenizer(teste_tokenizacao)
print(teste_ja_tokenizado)

['Estamos', 'fazendo', 'um', 'projeto', 'pro', 'BTG', '!', 'Ele', 'é', 'muito', 'interessante', '.']


In [ ]:
teste_tokenizacao2 = "Estamos fazendo um projeto pro BTG! ! Ele é muito interessante."
teste_ja_tokenizado2 = tokenizer(teste_tokenizacao2)
print(teste_ja_tokenizado2)

['Estamos', 'fazendo', 'um', 'projeto', 'pro', 'BTG', '!', '!', 'Ele', 'é', 'muito', 'interessante', '.']


In [ ]:
teste_tokenizacao3 = "Estamos fazendo um projeto pro BTG!! Ele é muito interessante."
teste_ja_tokenizado3 = tokenizer(teste_tokenizacao3)
print(teste_ja_tokenizado3)

['Estamos', 'fazendo', 'um', 'projeto', 'pro', 'BTG', '!', '!', 'Ele', 'é', 'muito', 'interessante', '.']


In [ ]:
teste_tokenizacao4 = "!Estamos fazendo um projeto pro BTG! Ele é muito interessante."
teste_ja_tokenizado4 = tokenizer(teste_tokenizacao4)
print(teste_ja_tokenizado4)

['!', 'Estamos', 'fazendo', 'um', 'projeto', 'pro', 'BTG', '!', 'Ele', 'é', 'muito', 'interessante', '.']


#### 5.2 Tratamento de emoji

#### 5.3 Remoção de stopwords

In [ ]:
teste_sem_stop_words = remove_stopwords(teste_ja_tokenizado)
print(teste_sem_stop_words)

['fazendo', 'projeto', 'pro', 'btg', '!', 'interessante', '.']


In [ ]:
teste_sem_stop_words2 = remove_stopwords(teste_ja_tokenizado2)
print(teste_sem_stop_words2)

['fazendo', 'projeto', 'pro', 'btg', '!', '!', 'interessante', '.']


In [ ]:
teste_sem_stop_words3 = remove_stopwords(teste_ja_tokenizado3)
print(teste_sem_stop_words3)

['fazendo', 'projeto', 'pro', 'btg', '!', '!', 'interessante', '.']


In [ ]:
teste_sem_stop_words4 = remove_stopwords(teste_ja_tokenizado4)
print(teste_sem_stop_words4)

['!', 'fazendo', 'projeto', 'pro', 'btg', '!', 'interessante', '.']


#### 5.4 Remoção de acentos


In [ ]:
teste_sem_acento = remover_pontuacao(teste_sem_stop_words)
print(teste_sem_acento)

['fazendo', 'projeto', 'pro', 'btg', 'interessante']


#### 5.5 Lematização


In [ ]:
teste_com_lematizacao = ['meninas', 'elefantes', 'portugues', 'running']
lemmatized_tokens = lemmatize_text(teste_com_lematizacao)
print(lemmatized_tokens)

['menina', 'elefante', 'portugue', 'running']


# 6. Processamento / Demonstrações

Nessa seção de processamento, nós aplicamos as funções criadas anteriormente e aplicamos elas no dataframe, como forma de demonstrar os resultados efetivos.
- tokenized = coluna tokenizada
- no_stopwords = coluna tokenizada e sem stopwords
- texto_processed = coluna tokenizada, sem stopwords e sem acentuações
- stemmed = coluna tokenizada, sem stopwords, sem acentuações e com o stemming aplicado

In [ ]:
data_limpo['tokenized'] = data_limpo['texto'].apply(tokenizer)
data_limpo 

<ipython-input-91-ae72bb018440>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_limpo['tokenized'] = data_limpo['texto'].apply(tokenizer)


,dataPublicada,autor,texto,sentimento,tipoInteracao,tokenized
0,"""2022-03-04 09:38:00""",v8_capital,Confira os resultados dos nossos fundos no mês...,NEUTRAL,marcação,"[Confira, os, resultados, dos, nossos, fundos,..."
1,"""2022-03-04 09:29:00""",winthegame_of,A Alvarez & Marsal estará conosco no Sportainm...,NEUTRAL,marcação,"[A, Alvarez, &, Marsal, estará, conosco, no, S..."
2,"""2022-03-04 09:20:00""",marta_bego,#Repost btgpactual with make_repost ・・・ Entend...,NEUTRAL,marcação,"[#, Repost, btgpactual, with, make_repost, ・・・..."
3,"""2022-03-04 08:36:00""",lmviapiana,Minuto touro de ouro,POSITIVE,comentário,"[Minuto, touro, de, ouro]"
4,"""2022-03-03 21:10:00""",vanilson_dos,@ricktolledo Sim,NEUTRAL,resposta,"[@, ricktolledo, Sim]"
...,...,...,...,...,...,...
12350,"""2022-11-30 11:51:00""",onixcapital,Uma noite de encontros e muito conhecimento on...,NEUTRAL,marcação,"[Uma, noite, de, encontros, e, muito, conhecim..."
12351,"""2022-11-30 12:25:00""",amgcapitalinvest,Erros financeiros para eliminar antes dos 30 a...,NEUTRAL,marcação,"[Erros, financeiros, para, eliminar, antes, do..."
12352,"""2022-11-30 12:18:00""",jayipacderota,Estou muito grato por todo o esforço que você ...,POSITIVE,comentário,"[Estou, muito, grato, por, todo, o, esforço, q..."
12353,"""2022-11-30 12:50:00""",moinho_cultural,"""👉GUADAKAN está chegando!!!! A aula pública e...",NEUTRAL,marcação,"[``, 👉GUADAKAN, está, chegando, !, !, !, !, A,..."


In [ ]:
data_limpo['no_stopwords'] = data_limpo['tokenized'].apply(remove_stopwords)
data_limpo

<ipython-input-92-2fab3eaca743>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_limpo['no_stopwords'] = data_limpo['tokenized'].apply(remove_stopwords)


,dataPublicada,autor,texto,sentimento,tipoInteracao,tokenized,no_stopwords
0,"""2022-03-04 09:38:00""",v8_capital,Confira os resultados dos nossos fundos no mês...,NEUTRAL,marcação,"[Confira, os, resultados, dos, nossos, fundos,...","[confira, resultados, fundos, mês, fevereiro/2..."
1,"""2022-03-04 09:29:00""",winthegame_of,A Alvarez & Marsal estará conosco no Sportainm...,NEUTRAL,marcação,"[A, Alvarez, &, Marsal, estará, conosco, no, S...","[alvarez, &, marsal, estará, conosco, sportain..."
2,"""2022-03-04 09:20:00""",marta_bego,#Repost btgpactual with make_repost ・・・ Entend...,NEUTRAL,marcação,"[#, Repost, btgpactual, with, make_repost, ・・・...","[#, repost, btgpactual, with, make_repost, ・・・..."
3,"""2022-03-04 08:36:00""",lmviapiana,Minuto touro de ouro,POSITIVE,comentário,"[Minuto, touro, de, ouro]","[minuto, touro, ouro]"
4,"""2022-03-03 21:10:00""",vanilson_dos,@ricktolledo Sim,NEUTRAL,resposta,"[@, ricktolledo, Sim]","[@, ricktolledo, sim]"
...,...,...,...,...,...,...,...
12350,"""2022-11-30 11:51:00""",onixcapital,Uma noite de encontros e muito conhecimento on...,NEUTRAL,marcação,"[Uma, noite, de, encontros, e, muito, conhecim...","[noite, encontros, conhecimento, onde, cliente..."
12351,"""2022-11-30 12:25:00""",amgcapitalinvest,Erros financeiros para eliminar antes dos 30 a...,NEUTRAL,marcação,"[Erros, financeiros, para, eliminar, antes, do...","[erros, financeiros, eliminar, antes, 30, anos..."
12352,"""2022-11-30 12:18:00""",jayipacderota,Estou muito grato por todo o esforço que você ...,POSITIVE,comentário,"[Estou, muito, grato, por, todo, o, esforço, q...","[grato, todo, esforço, rendeu, investimento, r..."
12353,"""2022-11-30 12:50:00""",moinho_cultural,"""👉GUADAKAN está chegando!!!! A aula pública e...",NEUTRAL,marcação,"[``, 👉GUADAKAN, está, chegando, !, !, !, !, A,...","[``, 👉guadakan, chegando, !, !, !, !, aula, pú..."


In [ ]:
data_limpo['texto_processed'] = data_limpo['no_stopwords'].apply(remover_pontuacao)
data_limpo

<ipython-input-93-272a836fa121>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_limpo['texto_processed'] = data_limpo['no_stopwords'].apply(remover_pontuacao)


,dataPublicada,autor,texto,sentimento,tipoInteracao,tokenized,no_stopwords,texto_processed
0,"""2022-03-04 09:38:00""",v8_capital,Confira os resultados dos nossos fundos no mês...,NEUTRAL,marcação,"[Confira, os, resultados, dos, nossos, fundos,...","[confira, resultados, fundos, mês, fevereiro/2...","[confira, resultados, fundos, mês, fevereiro22..."
1,"""2022-03-04 09:29:00""",winthegame_of,A Alvarez & Marsal estará conosco no Sportainm...,NEUTRAL,marcação,"[A, Alvarez, &, Marsal, estará, conosco, no, S...","[alvarez, &, marsal, estará, conosco, sportain...","[alvarez, marsal, estará, conosco, sportainmet..."
2,"""2022-03-04 09:20:00""",marta_bego,#Repost btgpactual with make_repost ・・・ Entend...,NEUTRAL,marcação,"[#, Repost, btgpactual, with, make_repost, ・・・...","[#, repost, btgpactual, with, make_repost, ・・・...","[repost, btgpactual, with, make_repost, entend..."
3,"""2022-03-04 08:36:00""",lmviapiana,Minuto touro de ouro,POSITIVE,comentário,"[Minuto, touro, de, ouro]","[minuto, touro, ouro]","[minuto, touro, ouro]"
4,"""2022-03-03 21:10:00""",vanilson_dos,@ricktolledo Sim,NEUTRAL,resposta,"[@, ricktolledo, Sim]","[@, ricktolledo, sim]","[ricktolledo, sim]"
...,...,...,...,...,...,...,...,...
12350,"""2022-11-30 11:51:00""",onixcapital,Uma noite de encontros e muito conhecimento on...,NEUTRAL,marcação,"[Uma, noite, de, encontros, e, muito, conhecim...","[noite, encontros, conhecimento, onde, cliente...","[noite, encontros, conhecimento, onde, cliente..."
12351,"""2022-11-30 12:25:00""",amgcapitalinvest,Erros financeiros para eliminar antes dos 30 a...,NEUTRAL,marcação,"[Erros, financeiros, para, eliminar, antes, do...","[erros, financeiros, eliminar, antes, 30, anos...","[erros, financeiros, eliminar, antes, 30, anos..."
12352,"""2022-11-30 12:18:00""",jayipacderota,Estou muito grato por todo o esforço que você ...,POSITIVE,comentário,"[Estou, muito, grato, por, todo, o, esforço, q...","[grato, todo, esforço, rendeu, investimento, r...","[grato, todo, esforço, rendeu, investimento, r..."
12353,"""2022-11-30 12:50:00""",moinho_cultural,"""👉GUADAKAN está chegando!!!! A aula pública e...",NEUTRAL,marcação,"[``, 👉GUADAKAN, está, chegando, !, !, !, !, A,...","[``, 👉guadakan, chegando, !, !, !, !, aula, pú...","[guadakan, chegando, aula, pública, formato, e..."


In [ ]:
# data_limpo['stemmed'] = data_limpo['texto_processed'].apply(stemming)
# data_limpo

# 7. Pipeline simples de pré-processamento - **IMPORTANTE**

Essa seção é extremamente importante, pois nela criamos a função de pipeline com todas as outras funções que criamos anteriormente. Então, ela é feita para que seja possível de rodar somente ela e ter um resultado efetivo!

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Módulo 6/projeto/base_nova_csv.csv')

FileNotFoundError: ignored

In [ ]:
data = data.rename(columns={'"anomalia"' : 'anomalia', '"dataPublicada"' : 'dataPublicada', '"autor"' : 'autor', '"texto"' : 'texto', '"sentimento"' : 'sentimento', '"tipoInteracao"' : 'tipoInteracao', '"probabilidadeAnomalia"' : 'probabilidadeAnomalia', '"linkPost"' : 'linkPost', '"processado"' : 'processado',  '"contemHyperlink"' : 'contemHyperlink' })

In [ ]:
data_dropado = data.drop(['processado', 'id', 'dataPublicada'], axis=1)

In [ ]:
data_limpo = data_dropado.loc[data_dropado['autor'] != 'btgpactual']

In [ ]:
data_limpo

## 7.1 Definição de Função

In [ ]:
def tokenizer(comment):
    if isinstance(comment, str):
        tokens = nltk.word_tokenize(comment)
        return tokens
    else:
        return []

In [ ]:
def remove_stopwords(tokens):
    if isinstance(tokens, list): 
        comments_filtered = []
        for token in tokens:
          tk = token.lower()
          if tk not in stopwords:
              comments_filtered.append(tk)
        return comments_filtered
    else:
        return []

In [ ]:
def remover_pontuacao(tokens):
    tokens_sem_pontuacao = []
    for token in tokens:
        token_sem_pontuacao = re.sub(r'[^\w\s]', '', token)
        if token_sem_pontuacao != '':
            tokens_sem_pontuacao.append(token_sem_pontuacao)
    return tokens_sem_pontuacao

In [ ]:
sn_stemmer = SnowballStemmer('portuguese')

def stemming(comments_no_accent):
    if isinstance(comments_no_accent, list):
        stem_list = []
        for token in comments_no_accent: 
            stem_list.append(sn_stemmer.stem(token))
        return stem_list
    else:
        return []

In [ ]:
def pipeline(comment):
      # Tokenization
      tokens = tokenizer(comment)
      # Removal of stopwords
      tokens_filtered = remove_stopwords(tokens)
      # Removal of punctuation
      tokens_no_punct = remover_pontuacao(tokens_filtered)
      # Stemming
      stemmed_tokens = stemming(tokens_no_punct)
      return stemmed_tokens

## 7.2 Testes da Função

In [ ]:
data_limpo['texto']

- Criação da nova coluna, somente com a função pipeline, que repassa todo o nosso pré-processamento.

In [ ]:
data_limpo['texto_tratado'] = data_limpo['texto'].apply(pipeline)

In [ ]:
data_limpo

---

In [ ]:
comment = "Estamos fazendo um projeto pro BTG!"
preprocessed_comment = pipeline(comment)
print(preprocessed_comment)

- Caso queira fazer o teste de como está o pré-processamento, digite uma frase!
- Descomente a célula de código abaixo para testar!
  - Selecione todo o conteúdo com seu cursor e clique 'CTRL + ; ' (ctrl + botão de ponto e vírgula)

In [ ]:
# comment = input("Digite uma frase da sua escolha e visualize o tratamento: ")
# preprocessed_comment = pipeline(comment)
# print(preprocessed_comment)

# 8. Bag of Words (BoW)

O modelo Bag of Words (BoW) é uma técnica utilizada em processamento de linguagem natural para representar um texto como um conjunto de palavras desordenadas, ignorando a ordem e a estrutura gramatical das frases. 

Nesse modelo, cada palavra única do texto é transformada em uma "feature" (característica), e a frequência de cada palavra no texto é usada como um valor numérico para a feature correspondente.

Por exemplo, a frase "O gato preto pulou o muro" seria representada como um conjunto de palavras desordenadas: `'o', 'gato', 'preto', 'pulou', 'o', 'muro'`. A frequência de cada palavra seria contada, e o resultado seria um vetor numérico que representa a frequência de cada palavra na frase.


In [ ]:
# Multiple documents
text = ["Estamos fazendo um projeto pro BTG!", "Somos alunos de Sistemas de Informação do Inteli", "O Renato é o nosso orientador", "O Hayashi é o nosso professor de programação"] 

# create the transform
vectorizer = CountVectorizer()

# tokenize and build vocab
vectorizer.fit(text)

# summarize
print(sorted(vectorizer.vocabulary_))

# encode document
vector = vectorizer.transform(text)

# summarize encoded vector
print(vector.shape)
print(vector.toarray())

**AVISO**:
O modelo Bag of Words é uma técnica simples e eficiente para representar textos em formato vetorial, o que permite utilizá-los em algoritmos de aprendizado de máquina. 

No entanto, essa abordagem ignora informações importantes sobre a estrutura e o significado das frases, como a ordem das palavras e as relações sintáticas entre elas. Por isso, é comum utilizar técnicas mais avançadas, como a modelagem de tópicos (topic modeling) e as redes neurais, para lidar com textos mais complexos.

Porém, para vieses acadêmicos, estamos implementando e aprendendo como é o funcionamento desse processo.

## 8.1 Definição da Função


In [ ]:
data_limpo['texto_tratado'] # chamando somente a coluna do dataframe, que ja foi tratada

In [ ]:
textos = data_limpo['texto_tratado']

In [ ]:
# Criando a função de vetorização, a qual irá criar um DataFrame com todos os termos e suas contabilizações em cada comentário (representados pelo index)
def bow(comentarios): 
     # Inicializa o CountVectorizer
    vectorizer = CountVectorizer(analyzer=lambda x: x)
    # Cria o modelo Bag of Words
    bow_model = vectorizer.fit_transform(comentarios)
    # Cria um dataframe com as palavras e suas frequências
    bow_df = pd.DataFrame(bow_model.toarray(), columns=vectorizer.get_feature_names_out())
    return bow_df

---

## 8.2 Testes das funções


Como é possível visualizar, temos uma tabela com todos os termos que foram considerados chaves para a análise.

In [ ]:
df = bow(textos)
df

- Mostrando a primeira palavra, do primeiro comentário, e como ela se repete uma vez e duas vezes em cada um dos outros comentários

In [ ]:
df['conf'].value_counts()

In [ ]:
df[df['conf'] == 2]

In [ ]:
word_counts = df.sum()
top_words = word_counts.sort_values(ascending=False)
top_10 = top_words.head(10)

In [ ]:
top_10

In [ ]:
palavras = list(top_words.index)
wordcloud = WordCloud(width=800, height=800, background_color='white').generate(' '.join(palavras))
plt.figure(figsize=(8,8), facecolor=None)
plt.imshow(wordcloud)
plt.axis("off")
plt.tight_layout(pad=0)
plt.show()

# 9. Plotagem de Gráficos

## 9.1 Autores que mais comentam

In [ ]:
# Contagem de palavras
autor_counts = data_limpo['autor'].explode().value_counts()

# Plotagem do gráfico de barras
plt.figure(figsize=(10, 6))
autor_counts.head(20).plot(kind='bar')
plt.xlabel('Autores')
plt.ylabel('Contagem')
plt.title('Autores que mais comentam')
plt.show()

## 9.2 Palavras mais frequentes

In [ ]:
# Contagem de palavras
word_counts = data_limpo['texto_tratado'].explode().value_counts()

# Plotagem do gráfico de barras
plt.figure(figsize=(10, 6))
word_counts.head(20).plot(kind='bar')
plt.xlabel('Palavra')
plt.ylabel('Contagem')
plt.title('Top 20 Palavras Mais Frequentes')
plt.show()

## 9.3 Tipo de interação

In [ ]:
# Contar os tipos de interação
count_interactions = data_limpo['tipoInteracao'].value_counts()
# Criar o gráfico de pizza
plt.figure(figsize=(8, 6))
count_interactions.plot(kind='pie', autopct='%1.1f%%')
plt.title('Tipos de Interação')
plt.ylabel('')
plt.show()

## 9.4 Classificação de sentimentos

In [ ]:
# Contar os tipos de sentimentos
count_sentimentos = data_limpo['sentimento'].value_counts()
# Criar o gráfico de pizza
plt.figure(figsize=(8, 6))
count_sentimentos.plot(kind='pie', autopct='%1.1f%%')
plt.title('Tipos de Sentimento')
plt.ylabel('')
plt.show()

In [ ]:
# Contar os tipos de sentimentos
count_sentimentos = data_limpo['sentimento'].value_counts()

# Plotagem do gráfico de barras
plt.figure(figsize=(10, 6))
count_sentimentos.plot(kind='bar')
plt.xlabel('Sentimentos')
plt.ylabel('Contagem')
plt.title('Tipos de Sentimento')
plt.show()

# 10. Representação vetorial de textos

In [ ]:
data_limpo

In [ ]:
import pandas as pd
from gensim.utils import simple_preprocess
# Acessar a coluna que contém o texto do corpus
text_column = data_limpo['texto']  # Substitua 'nome_da_coluna_de_texto' pelo nome correto da coluna
# Pré-processamento de cada documento do corpus
corpus_preprocessed = []
for doc in text_column:
    corpus_preprocessed.append(simple_preprocess(doc, deacc=True, min_len=3))
# Exemplo de visualização dos documentos pré-processados
for i, doc in enumerate(corpus_preprocessed):
    print(f"Documento {i+1}: {doc}")

In [ ]:
# Exemplo 2 – Vocabulário criado a partir dos documentos tokenizados e tratados

from gensim.corpora import Dictionary as corpDict
# nova função importada

vocab = corpDict(corpus_preprocessed)
print(vocab)

df2 = pd.DataFrame(vocab.values(), columns=['Palavra'])
display(df2)

In [ ]:
# Exemplo 3 – Treinamento de um modelo de word2vec

from gensim.models import Word2Vec

# Treina um modelo de Word2Vec
model = Word2Vec(corpus_preprocessed, min_count=1, vector_size=10)

# Exibe uma tabela com os vetores
dic = {v: model.wv[v] for v in vocab.values()}

df3 = pd.DataFrame(dic)
display(df3)

In [ ]:
# Exemplo 4 – Armazenando um modelo de word2vec

import gensim
import nltk
from gensim.models import Word2Vec

model_btg3 = gensim.models.Word2Vec(corpus_preprocessed, min_count=1)

model_btg3.save('modelo_btg3')

In [ ]:
# Exemplo 5 – Carregando um modelo
from gensim.models import Word2Vec

model_carreg = Word2Vec.load('modelo_btg3')

In [ ]:
# Exemplo 6 – Carregando um modelo pré-treinado
# import gensim.downloader as corpus_data

# model = corpus_data.load('word2vec-google-news-300')

In [ ]:
# # Exemplo 7 – Palavras similares a car e house

# dfC = pd.DataFrame(model_carreg.similar_by_word('beta', topn=10), columns=['Palavra', 'Similaridade'])
# dfC = dfC.set_index('Palavra')
# display(dfC)

# dfH = pd.DataFrame(model_carreg.similar_by_word('btgpactual', topn=10), columns=['Palavra', 'Similaridade'])
# dfH = dfH.set_index('Palavra')
# display(dfH)

In [ ]:
from gensim.models import Word2Vec
import pandas as pd

# Obtenha os tokens da coluna
tokenized_data = [tokens for tokens in data_limpo['no_stopwords']]

# Treine o modelo Word2Vec
model1 = Word2Vec(tokenized_data, min_count=1)

# Exemplo de uso do modelo treinado
word_vector = model.wv['btgpactual']
similar_words = model.wv.most_similar('btgpactual')

# Exiba os resultados
print("Vetor da palavra 'btgpactual':", word_vector)
print("Palavras similares a 'btgpactual':", similar_words)


# **Desenvolvido por todos os integrantes do grupo**
- Daniel Barzilai
- Larissa Carvalho
- Maria Luisa Maia
- Pedro Rezende
- Rafael Moritz
- Vitor Oliveira
